# 4) Dictionaries — Exercises

**Learning goals:** CRUD, iteration patterns, safe access (`get`, `setdefault`), merging, grouping, transforming, nested access.

### Warm-ups

1. **Make profile with defaults**

```python
def make_profile(name, email=None, country="IN"):
    ...
p = make_profile("A","a@example.com")
assert p["name"]=="A" and p["country"]=="IN"
```

2. **Safe get path** — nested dict path or default

```python
def get_in(d, path, default=None):
    ...
cfg = {"db":{"host":"localhost","port":5432}}
assert get_in(cfg, ["db","port"]) == 5432
assert get_in(cfg, ["db","user"], "postgres") == "postgres"
```

3. **Set with default container**

```python
def multimap_add(mm, key, value):
    """mm: dict key -> list; append value; create list if missing."""
    ...
mm = {}
multimap_add(mm,"a",1); multimap_add(mm,"a",2)
assert mm == {"a":[1,2]}
```

### Core

4. **Invert mapping (values unique)**
   If duplicate values appear, keep first key.

```python
def invert_unique(d):
    ...
assert invert_unique({"a":1,"b":2}) == {1:"a",2:"b"}
```

5. **Invert to sets (values may repeat)**

```python
def invert_to_sets(d):
    ...
assert invert_to_sets({"a":1,"b":1,"c":2}) == {1:{"a","b"}, 2:{"c"}}
```

6. **Merge with priority** — `override` wins

```python
def merge_priority(base, override):
    ...
assert merge_priority({"a":1,"b":2},{"b":9,"c":3}) == {"a":1,"b":9,"c":3}
```

7. **Group by key function**

```python
def group_by(xs, key_fn):
    ...
g = group_by(["apple","pear","plum","kiwi"], len)
assert g == {5:["apple"],4:["pear","plum","kiwi"]}
```

8. **Top-N by value**

```python
def top_n(d, n):
    ...
assert top_n({"a":3,"b":1,"c":5},2) == [("c",5),("a",3)]
```

9. **Join two “tables” by key** (left join)

```python
def left_join(left_rows, right_rows, key):
    """
    left_rows/right_rows: list of dicts
    Return: list of merged dicts; right fields may be missing.
    """
    ...
L = [{"id":1,"name":"A"},{"id":2,"name":"B"}]
R = [{"id":1,"score":90}]
out = left_join(L,R,"id")
assert out[0]["score"]==90 and out[1].get("score") is None
```

### Challenge

10. **Flatten nested dict with dot paths**

```python
def flatten_dot(d, parent_key=""):
    """
    {"a":{"b":1},"c":2} -> {"a.b":1, "c":2}
    """
    ...
assert flatten_dot({"a":{"b":1},"c":2}) == {"a.b":1,"c":2}
```
